# Notebook config

In [ ]:
#@markdown #**GOOGLE DRIVE - RGB, RedEdge y NDVI**
#@markdown ---

# Cell 0: ENVIRONMENT DETECTION & CONFIGURATION
import os
import sys
import subprocess

# Detect if running in Colab
RUNNING_IN_COLAB = 'google.colab' in sys.modules

if RUNNING_IN_COLAB:
    BASE_DIR = '/content'
    # Clone the RatTrack repository if it's not already present
    REPO_URL = 'https://github.com/leomaestri/RatTrack.git'
    REPO_DIR = os.path.join(BASE_DIR, 'RatTrack')
    if not os.path.exists(REPO_DIR):
        subprocess.run(['git', 'clone', REPO_URL, REPO_DIR], check=True)
    # Default dataset path from the repo
    !unzip -o {REPO_DIR}/data/dataset.zip -d {REPO_DIR}/data/dataset/
    DATA_DIR = os.path.join(REPO_DIR, 'data', 'dataset')
else:
    #@markdown **Ignore if using colab**. Name of the directory where the RatTrack repository is saved
    proyect_name = "" #@param {type:"string"}
    BASE_DIR = os.path.expanduser('~/proyecto')
    # Default dataset path locally (assuming you cloned RatTrack into ~/proyecto)
    !unzip -o {BASE_DIR}/data/dataset.zip -d {BASE_DIR}/data/dataset/
    DATA_DIR = os.path.join(BASE_DIR, 'data', 'dataset')

!pip install -r {REPO_DIR}/requirements.txt

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

%pip install optuna

In [ ]:
import cv2
import glob
import yaml
import random
import shutil
import optuna
import numpy as np
from ultralytics import YOLO

# Load dataset

Dataset must be in YOLOv11 format:
* dataset directory
 * images
   * train
   * val
 * labels
   * train
   * val
 * masks
   * train
   * val

Save dasaset to "/content/dataset"

In [ ]:
# If loading from google drive

#from google.colab import drive
#drive.mount('/content/drive')
#!cp -r "/content/drive/MyDrive/..." "/content/dataset" # Change ... for the dataset path in google drive

In [ ]:
# Directory containing the images to replace
src_dir = DATA_DIR + '/images/train'

# List only image files
files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.jpg', '.png'))]

# Shuffle and calculate half
random.shuffle(files)
n = len(files) // 2

for idx, fname in enumerate(files):
    path = os.path.join(src_dir, fname)
    img = cv2.imread(path)
    if img is None:
        continue  # skip if the image couldn't be read

    if idx < n:
        # Convert to infrared: R channel = gray, B/G = 0
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_ir = cv2.merge([np.zeros_like(gray), np.zeros_like(gray), gray])
        out = img_ir
    else:
        # Keep the original
        out = img

    # Overwrite the original image
    cv2.imwrite(path, out)

print(f"Processed {len(files)} images, changing to red light {n} of them")

In [ ]:
data = {
    'path': f"{DATA_DIR}",
    'train': 'images/train',
    'val':   'images/val',
    'nc': 1,
    'names': ['rat'],
    'mask': True
}
yaml_path = './data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)
print(f"data.yaml created on {yaml_path}")

# Hiperparameter Search

Do not run hyperparameter search if you are not going to change it. Just train directly

In [ ]:
# This is the last Optuna search after a couple of cycles.
# Feel free to keep looking for better hyperparameters, specially different Data Augmentations which where not changed much

n_trials=10
epochs=20

# Define the objective function for Optuna
def objective(trial):
    # 1) Suggest hyperparameters
    lr0           = trial.suggest_float('lr0', 3e-5, 2e-4, log=True)
    weight_decay  = 1.879e-05
    dropout       = 0.33

    # 2) Load a fresh model (reset state)
    model = YOLO('yolo11n-obb.pt')

    # 3) Quick training for evaluation (adjust epochs based on your hardware)
    model.train(
        data='./data.yaml',
        epochs=epochs,
        imgsz=640,
        lr0=lr0,
        weight_decay=weight_decay,
        dropout=dropout,
        verbose=False,
        patience=5,
        hsv_h=0.005, hsv_s=0.2, hsv_v=0.2,
        translate=0.1, scale=0.4,
        fliplr=0.5, flipud=0.5,
        mosaic=0.6, mixup=0.1,
        erasing=0.2, degrees=15.0,
        auto_augment=None,
        max_det=20

        #freeze=0,             # unfreeze all layers
        #mask_ratio=0,         # disable mask head
        #overlap_mask=False    # ignore mask overlaps
    )

    # 4) Evaluate on validation and return mAP@0.5 as the metric
    metrics = model.val(data='./data.yaml', imgsz=640, verbose=False)
    # return float(metrics.box.map50) # mAP at IoU=0.5
    return float(metrics.box.map)  # mAP at IoU=0.5–0.95

# 5) Run the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials)


[I 2025-07-01 23:40:06,599] A new study created in memory with name: no-name-cfbe2d87-4c57-46c8-862a-cbd7d9b050af


Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.3339393176448723, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=7.397982713331585e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-obb.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train14, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=T

train: Scanning /content/dataset_1/labels/train.cache... 750 images, 10 backgrounds, 0 corrupt: 100%|██████████| 760/760 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 757.0±560.5 MB/s, size: 126.9 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.49it/s]


                   all        150        150          1      0.998      0.995      0.952
Speed: 0.4ms preprocess, 4.9ms inference, 0.0ms loss, 8.5ms postprocess per image
Results saved to runs/obb/train28
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-obb summary (fused): 109 layers, 2,653,918 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1288.2±841.2 MB/s, size: 192.1 KB)


val: Scanning /content/dataset_1/labels/val.cache... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.80it/s]


                   all        150        150          1      0.998      0.995      0.951
Speed: 4.6ms preprocess, 6.7ms inference, 0.0ms loss, 8.0ms postprocess per image
Results saved to runs/obb/train282


[I 2025-07-02 01:01:20,824] Trial 14 finished with value: 0.995 and parameters: {'lr0': 1.2879195835332251e-05, 'weight_decay': 6.546302255495187e-06, 'dropout': 0.4460458887797003}. Best is trial 0 with value: 0.995.


In [ ]:
print("✅ Best trial:")
print(study.best_params)

# Train

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

lr0           = 7.4e-5
weight_decay  = 1.879e-05
dropout       = 0.33

model = YOLO('yolo11n-obb.pt')
model.train(
    data='./data.yaml',
    epochs=30,
    imgsz=640,
    patience=5,
    lr0=lr0,
    weight_decay=weight_decay,
    dropout=dropout,
    hsv_h=0.005, hsv_s=0.2, hsv_v=0.2,
    translate=0.1, scale=0.4,
    fliplr=0.5, flipud=0.5,
    mosaic=0.6, mixup=0.1,
    erasing=0.2, degrees=15.0,
    auto_augment=None,

    max_det=20
)

Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.33, dynamic=False, embed=None, epochs=30, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.005, hsv_s=0.2, hsv_v=0.2, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=7.4e-05, lrf=0.01, mask_ratio=4, max_det=20, mixup=0.1, mode=train, model=yolo11n-obb.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=train30, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrained=True, pr

train: Scanning /content/dataset_1/labels/train.cache... 750 images, 10 backgrounds, 0 corrupt: 100%|██████████| 760/760 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 897.8±665.9 MB/s, size: 126.9 KB)


val: Scanning /content/dataset_1/labels/val.cache... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]


...

                   all        150        150          1      0.999      0.995      0.949



30 epochs completed in 0.169 hours.
Optimizer stripped from runs/obb/train30/weights/last.pt, 5.7MB
Optimizer stripped from runs/obb/train30/weights/best.pt, 5.7MB

Validating runs/obb/train30/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-obb summary (fused): 109 layers, 2,653,918 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.46it/s]


                   all        150        150      0.999      0.993      0.995      0.955
Speed: 0.3ms preprocess, 4.4ms inference, 0.0ms loss, 7.7ms postprocess per image
Results saved to runs/obb/train30


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c1f5e67dc90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
!cp "./runs/obb/train30/weights/best.pt" "./best_model_from_train.pt"

print(f"Model saved to best_model_from_train.pt")